# Synthetix V3


In [8]:
import os
import time
from synthetix import Synthetix
from synthetix.utils import wei_to_ether, ether_to_wei, format_wei, format_ether
from synthetix.utils.multicall import call_erc7412
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
snx = Synthetix(
    provider_rpc=os.getenv("NETWORK_84532_RPC"),
    network_id=84532,
)

2024-06-13 09:39:06,561 - INFO - Using provided address without private key: 0x0000000000000000000000000000000000000000
2024-06-13 09:39:07,776 - WARNING - Failed to fetch core accounts: 0xb20f76e30000000000000000000000000000000000000000000000000000000000000000
2024-06-13 09:39:09,843 - WARNING - Failed to fetch perps accounts: 0xb20f76e30000000000000000000000000000000000000000000000000000000000000000
2024-06-13 09:39:11,134 - INFO - Fetching Pyth data for 32 markets


### Setup


In [10]:
# get some contracts
weth = snx.contracts["WETH"]["contract"]
usdc = snx.contracts["MintableToken"]["contract"]

In [11]:
def status():
    """A function printing some useful information about the connected account"""
    eth_balance = snx.get_eth_balance()
    susd_balance = snx.spot.get_balance(market_name="sUSD")
    susdc_balance = snx.spot.get_balance(market_name="sUSDC")

    usdc_balance = usdc.functions.balanceOf(snx.address).call()
    usdc_balance = format_wei(usdc_balance, 6)

    print(
        f"""
    Perps accounts: {snx.perps.account_ids}
    Core accounts: {snx.core.account_ids}
        
    ETH: {eth_balance['eth']}
    WETH ({weth.address}): {eth_balance['weth']}
    USDC ({usdc.address}): {usdc_balance}
    sUSDC: {susdc_balance}
    sUSD: {susd_balance}
    """
    )


status()


    Perps accounts: []
    Core accounts: []
        
    ETH: 23.700121824322625
    WETH (0x4200000000000000000000000000000000000006): 0.0
    USDC (0xc43708f8987Df3f3681801e5e640667D86Ce3C30): 0.0
    sUSDC: 0.0
    sUSD: 0.0
    


In [12]:
account_id = 170141183460469231731687303715884106043
BEFORE_BLOCK = 11220900

In [14]:
snx.perps.get_open_positions(account_id=account_id)

2024-06-13 09:39:20,515 - INFO - Using cached Pyth data


{'ETH': {'market_id': 100,
  'market_name': 'ETH',
  'pnl': 231.26724714914982,
  'accrued_funding': 115.53492177897475,
  'owed_interest': 0.000395768643723487,
  'position_size': -0.412}}

In [20]:
open_position = call_erc7412(
    snx,
    snx.perps.market_proxy,
    "getOpenPosition",
    (account_id, 100),
    block=BEFORE_BLOCK,
)

print(
    f"""
Pnl: {format_wei(open_position[0], 18)}
Accrued funding: {format_wei(open_position[1], 18)}
Position size: {format_wei(open_position[2], 18)}
Owed interest: {format_wei(open_position[3], 18)}
"""
)


Pnl: 166.12563143605132
Accrued funding: 108.43907493076163
Position size: -0.412
Owed interest: 0.000336966769097105



In [21]:
open_position = call_erc7412(
    snx, snx.perps.market_proxy, "getOpenPosition", (account_id, 100)
)

print(
    f"""
Pnl: {format_wei(open_position[0], 18)}
Accrued funding: {format_wei(open_position[1], 18)}
Position size: {format_wei(open_position[2], 18)}
Owed interest: {format_wei(open_position[3], 18)}
"""
)


Pnl: 232.25994781491707
Accrued funding: 115.54225766329078
Position size: -0.412
Owed interest: 0.000395591112488672



In [23]:
#########
# create a comparison for all of these calls as well:
# total_collateral_value = call_erc7412(
#     self.snx,
#     self.market_proxy,
#     "totalCollateralValue",
#     (account_id,),
#     calls=calls,
# )
# available_margin = call_erc7412(
#     self.snx,
#     self.market_proxy,
#     "getAvailableMargin",
#     (account_id,),
#     calls=calls,
# )
# withdrawable_margin = call_erc7412(
#     self.snx,
#     self.market_proxy,
#     "getWithdrawableMargin",
#     (account_id,),
#     calls=calls,
# )
# (
#     initial_margin_requirement,
#     maintenance_margin_requirement,
#     max_liquidation_reward,
# ) = call_erc7412(
#     self.snx,
#     self.market_proxy,
#     "getRequiredMargins",
#     (account_id,),
#     calls=calls,
# )
#########

collateral_before = call_erc7412(
    snx,
    snx.perps.market_proxy,
    "totalCollateralValue",
    (account_id,),
    block=BEFORE_BLOCK,
)
collateral_now = call_erc7412(
    snx, snx.perps.market_proxy, "totalCollateralValue", (account_id,)
)
available_margin_before = call_erc7412(
    snx,
    snx.perps.market_proxy,
    "getAvailableMargin",
    (account_id,),
    block=BEFORE_BLOCK,
)
available_margin_now = call_erc7412(
    snx, snx.perps.market_proxy, "getAvailableMargin", (account_id,)
)
withdrawable_margin_before = call_erc7412(
    snx,
    snx.perps.market_proxy,
    "getWithdrawableMargin",
    (account_id,),
    block=BEFORE_BLOCK,
)
withdrawable_margin_now = call_erc7412(
    snx, snx.perps.market_proxy, "getWithdrawableMargin", (account_id,)
)
(
    initial_margin_requirement,
    maintenance_margin_requirement,
    max_liquidation_reward,
) = call_erc7412(
    snx,
    snx.perps.market_proxy,
    "getRequiredMargins",
    (account_id,),
    block=BEFORE_BLOCK,
)
(
    initial_margin_requirement_now,
    maintenance_margin_requirement_now,
    max_liquidation_reward_now,
) = call_erc7412(snx, snx.perps.market_proxy, "getRequiredMargins", (account_id,))
print(
    f"""
Collateral before: {format_wei(collateral_before, 18)}
Collateral now: {format_wei(collateral_now, 18)}
Available margin before: {format_wei(available_margin_before, 18)}
Available margin now: {format_wei(available_margin_now, 18)}
Withdrawable margin before: {format_wei(withdrawable_margin_before, 18)}
Withdrawable margin now: {format_wei(withdrawable_margin_now, 18)}
Initial margin requirement before: {format_wei(initial_margin_requirement, 18)}
Initial margin requirement now: {format_wei(initial_margin_requirement_now, 18)}
Maintenance margin requirement before: {format_wei(maintenance_margin_requirement, 18)}
Maintenance margin requirement now: {format_wei(maintenance_margin_requirement_now, 18)}
Max liquidation reward before: {format_wei(max_liquidation_reward, 18)}
Max liquidation reward now: {format_wei(max_liquidation_reward_now, 18)}
"""
)


Collateral before: 439.5912759908266
Collateral now: 439.5912759908266
Available margin before: 605.7169074268779
Available margin now: 671.8662342611154
Withdrawable margin before: 521.0518176478926
Withdrawable margin now: 588.3702406551189
Initial margin requirement before: 84.6650897789853
Initial margin requirement now: 83.49616418133732
Maintenance margin requirement before: 63.30789757312795
Maintenance margin requirement now: 62.98907918067757
Max liquidation reward before: 5.002322826405647
Max liquidation reward now: 5.014101680421006

